## 训练U-Net模型

In [1]:
import json
import os

from types import SimpleNamespace
import tqdm
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

from dataset import BrainSegmentationDataset as Dataset
from logger import Logger
from loss import DiceLoss, LogCoshDiceLoss, ShapeAwareLoss
from transform import transforms
from unet import UNet
from nestedunet import NestedUNet
from utils import log_images, dsc

C:\Users\ext.yiwen\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 输入参数  
  
device: 设备编号  
batch_size: 批大小  
epochs: 训练轮数  
lr: 学习率  
vis_images: 可视化预测结果的数目 (在tensorboard中查看)  
vis_freq: 两次可视化预测结果的间隔  
weights: 训练后的模型参数路径    
images: 数据集路径   
image_size: 图像尺寸   
aug_scale: 数据增强(放缩)  
aug_angle: 数据增强(旋转)  

In [27]:
args = SimpleNamespace(
    device = 'cuda:0',
    batch_size = 20,
    epochs = 50,
    lr = 0.0001,
    workers = 0,
    vis_images = 200,
    vis_freq = 10,
    weights = './weights',
    logs = './logs_nestunet',
    images = './kaggle_3m',
    image_size = 256,
    aug_scale = 0.05,
    aug_angle = 15,
)

In [3]:
# 读取数据
def worker_init(worker_id):
    np.random.seed(42 + worker_id)

def data_loaders(args):
    dataset_train, dataset_valid = datasets(args)

    loader_train = DataLoader(
        dataset_train,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=args.workers,
        worker_init_fn=worker_init,
    )
    loader_valid = DataLoader(
        dataset_valid,
        batch_size=args.batch_size,
        drop_last=False,
        num_workers=args.workers,
        worker_init_fn=worker_init,
    )

    return dataset_train, dataset_valid, loader_train, loader_valid

# 数据集定义
def datasets(args):
    train = Dataset(
        images_dir=args.images,
        subset="train",
        image_size=args.image_size,
        transform=transforms(scale=args.aug_scale, angle=args.aug_angle, flip_prob=0.5),
    )
    valid = Dataset(
        images_dir=args.images,
        subset="validation",
        image_size=args.image_size,
        random_sampling=False,
    )
    return train, valid

# 数据处理
def dsc_per_volume(validation_pred, validation_true, patient_slice_index):
    dsc_list = []
    num_slices = np.bincount([p[0] for p in patient_slice_index])
    index = 0
    for p in range(len(num_slices)):
        y_pred = np.array(validation_pred[index : index + num_slices[p]])
        y_true = np.array(validation_true[index : index + num_slices[p]])
        dsc_list.append(dsc(y_pred, y_true))
        index += num_slices[p]
    return dsc_list


def log_loss_summary(logger, loss, step, prefix=""):
    logger.scalar_summary(prefix + "loss", np.mean(loss), step)


def makedirs(args):
    os.makedirs(args.weights, exist_ok=True)
    os.makedirs(args.logs, exist_ok=True)


def snapshotargs(args):
    args_file = os.path.join(args.logs, "args.json")
    with open(args_file, "w") as fp:
        json.dump(vars(args), fp)

In [28]:
makedirs(args)
snapshotargs(args)
device = torch.device("cpu" if not torch.cuda.is_available() else args.device)

#dataset_train, dataset_valid, loader_train, loader_valid = data_loaders(args)
#loaders = {"train": loader_train, "valid": loader_valid}


In [5]:
# save the data loader for pre-processed data to save future running time
torch.save(loader_train, 'train.pth')
torch.save(loader_valid, 'valid.pth')

In [6]:
# load the dataloader that has been saved out
loaders = {"train": torch.load('train.pth'), "valid": torch.load('valid.pth')}

In [29]:
#unet = UNet(in_channels=Dataset.in_channels, out_channels=Dataset.out_channels)
#unet.to(device)
nestedunet = NestedUNet(in_channels=Dataset.in_channels, out_channels=Dataset.out_channels)
nestedunet.to(device)

dsc_loss = DiceLoss()
# criterion = torch.nn.BCELoss()
# lcsce = LogCoshDiceLoss()
# shape_aware = ShapeAwareLoss()

best_validation_dsc = 0.0

# optimizer = optim.SGD(unet.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-4, nesterov=True)
optimizer = optim.Adam(nestedunet.parameters(), lr=args.lr)


logger = Logger(args.logs)
loss_train = []
loss_valid = []

step = 0


In [ ]:
for epoch in range(args.epochs):
    for phase in ["train", "valid"]:
        if phase == "train":
            nestedunet.train()
        else:
            nestedunet.eval()

        validation_pred = []
        validation_true = []

        for i, data in enumerate(tqdm.tqdm(loaders[phase])):
            if phase == "train":
                step += 1

            x, y_true = data
            x, y_true = x.to(device), y_true.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                y_pred = nestedunet(x)

                loss = dsc_loss(y_pred, y_true)
                #loss = criterion(y_pred, y_true)
                #loss = lcsce(y_pred, y_true)
                #loss = shape_aware(y_pred, y_true)

                if phase == "valid":
                    loss_valid.append(loss.item())
                    y_pred_np = y_pred.detach().cpu().numpy()
                    validation_pred.extend(
                        [y_pred_np[s] for s in range(y_pred_np.shape[0])]
                    )
                    y_true_np = y_true.detach().cpu().numpy()
                    validation_true.extend(
                        [y_true_np[s] for s in range(y_true_np.shape[0])]
                    )
                    if (epoch % args.vis_freq == 0) or (epoch == args.epochs - 1):
                        if i * args.batch_size < args.vis_images:
                            tag = "image/{}".format(i)
                            num_images = args.vis_images - i * args.batch_size
                            logger.image_list_summary(
                                tag,
                                log_images(x, y_true, y_pred)[:num_images],
                                step,
                            )

                if phase == "train":
                    loss_train.append(loss.item())
                    loss.backward()
                    optimizer.step()

            if phase == "train" and (step + 1) % 10 == 0:
                log_loss_summary(logger, loss_train, step)
                loss_train = []

        if phase == "valid":
            log_loss_summary(logger, loss_valid, step, prefix="val_")
            print("epoch {} | val_loss: {}".format(epoch + 1, np.mean(loss_valid)))
            mean_dsc = np.mean(
                dsc_per_volume(
                    validation_pred,
                    validation_true,
                    loaders["valid"].dataset.patient_slice_index,
                )
            )
            logger.scalar_summary("val_dsc", mean_dsc, step)
            print("epoch {} | val_dsc: {}".format(epoch+1, mean_dsc))
            if mean_dsc > best_validation_dsc:
                best_validation_dsc = mean_dsc
                # save weights out for inference
                torch.save(nestedunet.state_dict(), os.path.join(args.weights, "nestedunet.pt"))
            loss_valid = []

print("Best validation mean DSC: {:4f}".format(best_validation_dsc))

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:17<00:00,  1.02s/it]


epoch 1 | val_loss: 0.9202391259810504
epoch 1 | val_dsc: 0.7047440432966343


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  5.98it/s]


epoch 2 | val_loss: 0.9082802709411172
epoch 2 | val_dsc: 0.5891243867935535


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  6.13it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  6.31it/s]


epoch 43 | val_loss: 0.15672071190441356
epoch 43 | val_dsc: 0.907381908441096


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  6.26it/s]


epoch 44 | val_loss: 0.14658164978027344
epoch 44 | val_dsc: 0.9010459407766038


 98%|██████████████████████████████████████████████████████████████████████████████  | 162/166 [02:11<00:03,  1.20it/s]

In [11]:
# save the model out
torch.save(nestedunet.state_dict(), 'nestedunet-batch16-epoch100-lr00005-adam-LogCoshDiceLoss')